In [1]:
import json
import pandas as pd
from datetime import datetime
from listas import charlson_dict, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import (obtenir_data_presencia_codi, restar_dates, codis_ICD, nombre_ingressos,
                                 dies_ingressat_total, interval_10_edat,
                                 sumar_barthel, sumar_emina, obtenir_ultima_clau, obtenir_pes_o_mitjana,
                                 canadenca_comparada, disfagia_mecvvs, extreure_valors_claus,
                                 extreure_valors_claus_simple, obtenir_valors_clau_interes, index_charlson,
                                 obtenir_pes_mes_antic, obtenir_pes_mes_nou, obtenir_data_pes_mes_antic,
                                 obtenir_primera_data_mecvv, obtenir_pes_coincident_mecvv, restar_columnes_object)

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [2]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


In [3]:
AMB_PA = taula[taula["PA_diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)]
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) &
           (taula['P diagnosticada'] == 1.0)]
TOTAL = taula

KeyError: 'PA_diagnosticada'

# Síndromes geriàtrics

## Index de Charlson

In [ ]:
Mitjana_charlson = AMB_PA['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

In [ ]:
Mitjana_charlson = AMB_PA_MECVV['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA_MECVV['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

In [ ]:
Mitjana_charlson = SENSE_PA['Charlson'].mean()
desviacio_estandard_charlson = SENSE_PA['charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

In [ ]:
Mitjana_charlson = TOTAL['Charlson'].mean()
desviacio_estandard_charlson = TOTAL['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

#### Pvalor

##### Normalitat

In [ ]:
stat, p_value = shapiro(AMB_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

In [4]:
stat, p_value = shapiro(AMB_PA_MECVV['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [5]:
stat, p_value = shapiro(SENSE_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [6]:
stat, p_value = shapiro(TOTAL['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'TOTAL' is not defined

In [255]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Charlson'], AMB_PA_MECVV['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [257]:
stat, p_value_13 = ttest_ind(AMB_PA['Charlson'], SENSE_PA['Charlson'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [9]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Charlson'], SENSE_PA['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Caigudes prèvies

In [10]:
caprev_AMB_PA= (AMB_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [11]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [12]:
caprev_SENSE_PA= (SENSE_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [13]:
stat, p_value = shapiro(AMB_PA['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [14]:
stat, p_value = shapiro(AMB_PA_MECVV['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [15]:
stat, p_value = shapiro(SENSE_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [16]:
stat, p_value_12 = mannwhitneyu(AMB_PA['caidas_previas'], AMB_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [17]:
stat, p_value_13 = mannwhitneyu(AMB_PA['caidas_previas'], SENSE_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [18]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['caidas_previas'], SENSE_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Deliris

In [19]:
del_AMB_PA= (AMB_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [20]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [21]:
del_SENSE_PA= (SENSE_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [22]:
stat, p_value = shapiro(AMB_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [23]:
stat, p_value = shapiro(AMB_PA_MECVV['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [24]:
stat, p_value = shapiro(SENSE_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [25]:
stat, p_value_12 = mannwhitneyu(AMB_PA['delirios'], AMB_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [26]:
stat, p_value_13 = mannwhitneyu(AMB_PA['delirios'], SENSE_PA['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [27]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['delirios'], SENSE_PA['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Demència

In [28]:
dem_AMB_PA= (AMB_PA['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [29]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [30]:
dem_SENSE_PA= (SENSE_PA['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [31]:
stat, p_value = shapiro(AMB_PA['demencia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [32]:
stat, p_value = shapiro(AMB_PA_MECVV['demencia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [33]:
stat, p_value = shapiro(SENSE_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [34]:
stat, p_value_12 = mannwhitneyu(AMB_PA['demencia'], AMB_PA_MECVV['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [35]:
stat, p_value_13 = mannwhitneyu(AMB_PA['demencia'], SENSE_PA['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [36]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['demencia'], SENSE_PA['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Síndrome depressiu

In [37]:
depre_AMB_PA= (AMB_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [38]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [39]:
depre_SENSE_PA= (SENSE_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [40]:
stat, p_value = shapiro(AMB_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [41]:
stat, p_value = shapiro(AMB_PA_MECVV['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [42]:
stat, p_value = shapiro(SENSE_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [43]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sindrome_depresivo'], AMB_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [44]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sindrome_depresivo'], SENSE_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [45]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sindrome_depresivo'], SENSE_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Incontinència urinària

In [46]:
ic_AMB_PA= (AMB_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [47]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [48]:
ic_SENSE_PA= (SENSE_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [49]:
stat, p_value = shapiro(AMB_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [50]:
stat, p_value = shapiro(AMB_PA_MECVV['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [51]:
stat, p_value = shapiro(SENSE_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [52]:
stat, p_value_12 = mannwhitneyu(AMB_PA['incont_uri'], AMB_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [53]:
stat, p_value_13 = mannwhitneyu(AMB_PA['incont_uri'], SENSE_PA['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [54]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['incont_uri'], SENSE_PA['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Incontinència fecal 

In [55]:
ifec_AMB_PA= (AMB_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [56]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [57]:
ifec_SENSE_PA= (SENSE_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [58]:
stat, p_value = shapiro(AMB_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [59]:
stat, p_value = shapiro(AMB_PA_MECVV['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [60]:
stat, p_value = shapiro(SENSE_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [61]:
stat, p_value_12 = mannwhitneyu(AMB_PA['incont_fec'], AMB_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [62]:
stat, p_value_13 = mannwhitneyu(AMB_PA['incont_fec'], SENSE_PA['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [63]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['incont_fec'], SENSE_PA['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Úlceres de pressió

In [64]:
up_AMB_PA= (AMB_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [65]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [66]:
up_SENSE_PA= (SENSE_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [67]:
stat, p_value = shapiro(AMB_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [68]:
stat, p_value = shapiro(AMB_PA_MECVV['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [69]:
stat, p_value = shapiro(SENSE_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [70]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ulceras_presion'], AMB_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [71]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ulceras_presion'], SENSE_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [72]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ulceras_presion'], SENSE_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Immobilitat

In [73]:
imm_AMB_PA= (AMB_PA['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [74]:
imm_AMB_PA_MECVV= (AMB_PA_MECVV['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [75]:
imm_SENSE_PA= (SENSE_PA['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [76]:
stat, p_value = shapiro(AMB_PA['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [77]:
stat, p_value = shapiro(AMB_PA_MECVV['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [78]:
stat, p_value = shapiro(SENSE_PA['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [79]:
stat, p_value_12 = ttest_ind(AMB_PA['immobilitat'], AMB_PA_MECVV['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [80]:
stat, p_value_13 = ttest_ind(AMB_PA['immobilitat'], SENSE_PA['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [81]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['immobilitat'], SENSE_PA['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Confusió

In [82]:
conf_AMB_PA= (AMB_PA['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [83]:
conf_AMB_PA_MECVV= (AMB_PA_MECVV['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [84]:
conf_SENSE_PA= (SENSE_PA['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [85]:
stat, p_value = shapiro(AMB_PA['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [86]:
stat, p_value = shapiro(AMB_PA_MECVV['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [87]:
stat, p_value = shapiro(SENSE_PA['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [88]:
stat, p_value_12 = mannwhitneyu(AMB_PA['confusio'], AMB_PA_MECVV['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [89]:
stat, p_value_13 = mannwhitneyu(AMB_PA['confusio'], SENSE_PA['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [90]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['confusio'], SENSE_PA['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Osteoporosi

In [91]:
ost_AMB_PA= (AMB_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [92]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [93]:
ost_SENSE_PA= (SENSE_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [94]:
stat, p_value = shapiro(AMB_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [95]:
stat, p_value = shapiro(AMB_PA_MECVV['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [96]:
stat, p_value = shapiro(SENSE_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [97]:
stat, p_value_12 = ttest_ind(AMB_PA['osteoporosis'], AMB_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [98]:
stat, p_value_13 = ttest_ind(AMB_PA['osteoporosis'], SENSE_PA['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [99]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['osteoporosis'], SENSE_PA['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Sarcopènia

In [100]:
sarc_AMB_PA= (AMB_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [101]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [102]:
sarc_SENSE_PA= (SENSE_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [103]:
stat, p_value = shapiro(AMB_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [104]:
stat, p_value = shapiro(AMB_PA_MECVV['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [105]:
stat, p_value = shapiro(SENSE_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [106]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sarcopenia'], AMB_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [107]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [108]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Trastorns del son

In [109]:
ts_AMB_PA= (AMB_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [110]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [111]:
ts_SENSE_PA= (SENSE_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [112]:
stat, p_value = shapiro(AMB_PA['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [113]:
stat, p_value = shapiro(AMB_PA_MECVV['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [114]:
stat, p_value = shapiro(SENSE_PA['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [115]:
stat, p_value_12 = ttest_ind(AMB_PA['problsueño'], AMB_PA_MECVV['problsueño'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [116]:
stat, p_value_13 = ttest_ind(AMB_PA['problsueño'], SENSE_PA['problsueño'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [117]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['problsueño'], SENSE_PA['problsueño'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Dolor crònic

In [118]:
dc_AMB_PA= (AMB_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [119]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [120]:
dc_SENSE_PA= (SENSE_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [121]:
stat, p_value = shapiro(AMB_PA['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [122]:
stat, p_value = shapiro(AMB_PA_MECVV['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [123]:
stat, p_value = shapiro(SENSE_PA['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [124]:
stat, p_value_12 = ttest_ind(AMB_PA['dolor_cron'], AMB_PA_MECVV['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [125]:
stat, p_value_13 = ttest_ind(AMB_PA['dolor_cron'], SENSE_PA['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [126]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['dolor_cron'], SENSE_PA['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Iatrogènic

In [127]:
iatr_AMB_PA= (AMB_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [128]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [129]:
iatr_SENSE_PA= (SENSE_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [130]:
stat, p_value = shapiro(AMB_PA['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [131]:
stat, p_value = shapiro(AMB_PA_MECVV['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [132]:
stat, p_value = shapiro(SENSE_PA['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [133]:
stat, p_value_12 = ttest_ind(AMB_PA['iatrogenico'], AMB_PA_MECVV['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [134]:
stat, p_value_13 = ttest_ind(AMB_PA['iatrogenico'], SENSE_PA['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [135]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['iatrogenico'], SENSE_PA['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Restrenyiment

In [136]:
est_AMB_PA= (AMB_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [137]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [138]:
est_SENSE_PA= (SENSE_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [139]:
stat, p_value = shapiro(AMB_PA['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [140]:
stat, p_value = shapiro(AMB_PA_MECVV['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [141]:
stat, p_value = shapiro(SENSE_PA['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [142]:
stat, p_value_12 = ttest_ind(AMB_PA['estreñimiento'], AMB_PA_MECVV['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [143]:
stat, p_value_13 = ttest_ind(AMB_PA['estreñimiento'], SENSE_PA['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [144]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['estreñimiento'], SENSE_PA['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

# Historial mèdic/comorbilitats

## Malalties CV

In [145]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [146]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [147]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat 

In [148]:
stat, p_value = shapiro(AMB_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [149]:
stat, p_value = shapiro(AMB_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [150]:
stat, p_value = shapiro(SENSE_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [151]:
stat, p_value_12 = mannwhitneyu(AMB_PA['CV'], AMB_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [152]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [153]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Malalties del cor

In [154]:
cora_AMB_PA= (AMB_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [155]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [156]:
cora_SENSE_PA= (SENSE_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [157]:
stat, p_value = shapiro(AMB_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [158]:
stat, p_value = shapiro(AMB_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [159]:
stat, p_value = shapiro(SENSE_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [160]:
stat, p_value_12 = mannwhitneyu(AMB_PA['probl_corazon'], AMB_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [161]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [162]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Malalties neurodegeneratives

In [163]:
nd_AMB_PA= (AMB_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [164]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [165]:
nd_SENSE_PA= (SENSE_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [166]:
stat, p_value = shapiro(AMB_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [167]:
stat, p_value = shapiro(AMB_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [168]:
stat, p_value = shapiro(SENSE_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [169]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neurodegenerativas'], AMB_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [170]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [171]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Diabetes mellitus

In [172]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [173]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [174]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor 

##### Normalitat

In [175]:
stat, p_value = shapiro(AMB_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [176]:
stat, p_value = shapiro(AMB_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [177]:
stat, p_value = shapiro(SENSE_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [178]:
stat, p_value_12 = mannwhitneyu(AMB_PA['DM'], AMB_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [179]:
stat, p_value_13 = mannwhitneyu(AMB_PA['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PAN", p_value_13)

NameError: name 'AMB_PA' is not defined

In [180]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PAN", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Hepatopaties

In [181]:
hp_AMB_PA= (AMB_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [182]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [183]:
hp_SENSE_PA= (SENSE_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [184]:
stat, p_value = shapiro(AMB_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [185]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [186]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [187]:
stat, p_value_12 = mannwhitneyu(AMB_PA['hepatopatias'], AMB_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [188]:
stat, p_value_13 = mannwhitneyu(AMB_PA['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [189]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Neoplàsia

In [190]:
neopl_AMB_PA= (AMB_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [191]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [192]:
neopl_SENSE_PA= (SENSE_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [193]:
stat, p_value = shapiro(AMB_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [194]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [195]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [196]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neoplasias'], AMB_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [197]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_12)

NameError: name 'AMB_PA' is not defined

In [198]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [199]:
AMB_PA.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA['creatinina'], errors='coerce')
AMB_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA_MECVV['creatinina'], errors='coerce')
SENSE_PA.loc[:, 'creatinina'] = pd.to_numeric(SENSE_PA['creatinina'], errors='coerce')

NameError: name 'AMB_PA' is not defined

In [200]:
ecr_AMB_PA= (AMB_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [201]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [202]:
ecr_SENSE_PA= (SENSE_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [203]:
stat, p_value = shapiro(AMB_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [204]:
stat, p_value = shapiro(AMB_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [205]:
stat, p_value = shapiro(SENSE_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [206]:
stat, p_value_12 = ttest_ind(AMB_PA['creatinina'], AMB_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [207]:
stat, p_value_13 = mannwhitneyu(AMB_PA['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [208]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Fallida renal aguda (o insuficiència renal aguda)

In [209]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [210]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [211]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [212]:
stat, p_value = shapiro(AMB_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [213]:
stat, p_value = shapiro(AMB_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [214]:
stat, p_value = shapiro(SENSE_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [215]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ARF'], AMB_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [216]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [217]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Marejos/síncope/lipotímia/reacció vasovagal

In [218]:
dizsyn_AMB_PA= (AMB_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [219]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [220]:
dizsyn_SENSE_PA= (SENSE_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [221]:
stat, p_value = shapiro(AMB_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [222]:
stat, p_value = shapiro(AMB_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [223]:
stat, p_value = shapiro(SENSE_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [224]:
stat, p_value_12 = ttest_ind(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [225]:
stat, p_value_12 = mannwhitneyu(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [226]:
stat, p_value_13 = mannwhitneyu(AMB_PA['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [227]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## VIH

In [228]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [229]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [230]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [231]:
stat, p_value = shapiro(AMB_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [232]:
stat, p_value = shapiro(AMB_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [233]:
stat, p_value = shapiro(SENSE_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [234]:
stat, p_value_12 = ttest_ind(AMB_PA['VIH'], AMB_PA_MECVV['VIH'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [235]:
stat, p_value_13 = ttest_ind(AMB_PA['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [236]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Psicosi

In [237]:
psi_AMB_PA= (AMB_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [238]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [239]:
psi_SENSE_PA= (SENSE_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor 

##### Normalitat

In [240]:
stat, p_value = shapiro(AMB_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [241]:
stat, p_value = shapiro(AMB_PA_MECVV['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [242]:
stat, p_value = shapiro(SENSE_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [243]:
stat, p_value_12 = ttest_ind(AMB_PA['psicosis'], AMB_PA_MECVV['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [244]:
stat, p_value_13 = ttest_ind(AMB_PA['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [245]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined

## Deficiències nutricionals

In [246]:
nutri_AMB_PA= (AMB_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

NameError: name 'AMB_PA' is not defined

In [247]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

NameError: name 'AMB_PA_MECVV' is not defined

In [248]:
nutri_SENSE_PA= (SENSE_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

NameError: name 'SENSE_PA' is not defined

#### Pvalor

##### Normalitat

In [249]:
stat, p_value = shapiro(AMB_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA' is not defined

In [250]:
stat, p_value = shapiro(AMB_PA_MECVV['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'AMB_PA_MECVV' is not defined

In [251]:
stat, p_value = shapiro(SENSE_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

NameError: name 'SENSE_PA' is not defined

In [252]:
stat, p_value_12 = ttest_ind(AMB_PA['def_nutri'], AMB_PA_MECVV['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

NameError: name 'AMB_PA' is not defined

In [253]:
stat, p_value_13 = ttest_ind(AMB_PA['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

NameError: name 'AMB_PA' is not defined

In [254]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

NameError: name 'AMB_PA_MECVV' is not defined